In [240]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download("all")
from nltk.stem import *
from nltk.tokenize import *
from sklearn.feature_extraction.text import *

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/amith/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/amith/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/amith/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/amith/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /Users/amith/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to
[nltk_data]    |

In [241]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [242]:
df = pd.read_excel("IELTS_Success_Stories_Dataset.xlsx")

In [243]:
df

,Candidate,Location,Profession,Study Duration (months),IELTS Score (Overall),Key Strategies,Education Level,Age,Target Country,English Proficiency (Preparation),...,Preferred Learning Resources,Application Status,Job Offer Received,Additional Certifications,Volunteer Experience,Language Fluency,Internship Experience,Relevant Skills,Recommendations,Networking Efforts
0,Aisha,Karachi,Grad Student,3,8.500,"Practice, forums, tutoring",Master's,24,Canada,Advanced,...,"Online courses, study guides",Under Process,No,TESOL,Yes,"Urdu, English",Research Assistant,"Data Analysis, Teaching",Strong,"LinkedIn, Local Events"
1,Ahmed,Lahore,IT Professional,4,7.500,"Balanced schedule, time management, speaking p...",Bachelor's,29,Australia,Intermediate,...,"Online forums, practice tests",Approved,Yes,PMP,No,"Urdu, English",IT Consultant,"Project Management, Programming",Positive,Online Communities
2,Fatima,Islamabad,School Teacher,2,9.000,"Intensive course, online resources, mock inter...",Master's,32,New Zealand,Advanced,...,"Study guides, language exchange",Under Process,No,CELTA,Yes,"Urdu, English",Curriculum Developer,"Lesson Planning, Educational Tech",Positive,Professional Groups
3,Ali,Peshawar,IT Professional,5,8.000,"Focused study, workshops, peer feedback",Bachelor's,27,USA,Advanced,...,"Mock exams, speaking clubs",Rejected,No,AWS Certified Solutions Architect,No,"Urdu, English",Software Developer,"Cloud Computing, Programming",Strong,Online Platforms
4,Saad,Lahore,Eng. Student,6,8.250,"Consistent practice, private tutor, speaking c...",Bachelor's,22,UK,Advanced,...,"Online tutorials, speaking practice",Under Process,No,IELTS Examiner,Yes,"Urdu, English",Research Assistant,"Lab Techniques, Data Analysis",Positive,University Events
5,Nida,Islamabad,Marketing Manager,3,7.875,"Online courses, group study, extensive reading",Master's,30,Singapore,Intermediate,...,"Podcasts, business articles",Pending,No,Digital Marketing,No,"Urdu, English",Marketing Coordinator,"SEO, Content Creation",Neutral,Industry Conferences
6,Bilal,Karachi,Finance Analyst,4,7.000,"Self-study, vocabulary building, speaking prac...",Master's,28,UAE,Intermediate,...,"Financial news, language exchange",Pending,No,CFA Level I,No,"Urdu, English",Financial Analyst,"Financial Modeling, Data Analysis",Neutral,Online Forums
7,Sara,Lahore,Medical Student,5,8.750,"Daily practice, study groups, extensive reading",Bachelor's,23,Ireland,Advanced,...,"Medical journals, language exchange",Approved,No,BLS Certification,Yes,"Urdu, English",Research Assistant,"Lab Techniques, Data Analysis",Positive,Medical Conferences
8,Imran,Islamabad,Software Engineer,3,8.500,"Private tutor, mock tests, listening skill imp...",Bachelor's,26,Germany,Advanced,...,"Coding platforms, language exchange",Under Process,No,ISTQB Certified Tester,Yes,"Urdu, English",Software Developer,"Programming, QA Testing",Positive,Coding Communities
9,Hina,Karachi,Journalist,6,7.500,"Balanced study routine, writing practice, onli...",Master's,31,Malaysia,Intermediate,...,"News articles, language exchange",Rejected,No,Journalism Certification,No,"Urdu, English",Content Writer,"Writing, Editing",Neutral,Freelance Platforms


In [244]:
multiple_features = ["Key Strategies","Preferred Learning Resources","Relevant Skills"]
df_1 = df.copy()

col = df.columns.values
new_col = []
num_col = []
start = 0
for i in multiple_features:
    data = df[i].copy()
    d = np.array(list(map(lambda x: [i.strip() for i in x.split(",")],data)))
    colu = []
    for j in range(d.shape[1]):
        df_1[i+" "+str(j)] = d[:,j]
        colu.append(i+" "+str(j))
    df_1 = df_1.drop(columns=i)
    stop = np.where(col==i)[0][0]
    new_col += col[start:stop].tolist()
    new_col += colu
    num_col += colu
    start = stop + 1
df_2 = df_1[new_col].drop(columns=["Language Fluency"])
df_2["IELTS Score (Overall)"] = df_2["IELTS Score (Overall)"].round(2)

In [245]:
df_3 = df_2.copy()
for i in num_col:
    df_3[i] = df_2[i].str.lower()

In [246]:
key_strategies = []

key_strategies += df_3["Key Strategies 0"].unique().tolist()
key_strategies += df_3["Key Strategies 1"].unique().tolist()
key_strategies += df_3["Key Strategies 2"].unique().tolist()

key_strategies = np.sort(pd.Series(key_strategies).unique()).tolist()


preferred_learning = []

preferred_learning += df_3["Preferred Learning Resources 0"].unique().tolist()
preferred_learning += df_3["Preferred Learning Resources 1"].unique().tolist()

preferred_learning = np.sort(pd.Series(preferred_learning).unique()).tolist()


relevant_skills = []

relevant_skills += df_3["Relevant Skills 0"].unique().tolist()
relevant_skills += df_3["Relevant Skills 1"].unique().tolist()

relevant_skills = np.sort(pd.Series(relevant_skills).unique()).tolist()

In [247]:
print(key_strategies)

['balanced schedule', 'balanced study routine', 'consistent practice', 'daily practice', 'extensive reading', 'focused study', 'forums', 'group study', 'intensive course', 'intensive speaking practice', 'listening skill improvement', 'mock interviews', 'mock tests', 'online courses', 'online forums', 'online resources', 'peer feedback', 'practice', 'practice tests', 'private tutor', 'self-paced study', 'self-study', 'speaking clubs', 'speaking practice', 'speaking with native speakers', 'structured study plan', 'study groups', 'time management', 'tutoring', 'vocabulary building', 'vocabulary enhancement', 'workshops', 'writing practice', 'writing workshops']


In [303]:
tfidf = TfidfVectorizer()
ppp = tfidf.fit_transform(key_strategies)
qqq = cosine_similarity(ppp).round(2)

key_strategies_dict = {}
for i in key_strategies:
    hh = key_strategies.index(i)
    n = []
    for j in key_strategies[hh+1:]:
        ww = key_strategies.index(j)
        if qqq[hh,ww] > 0:
            n.append(j)
        key_strategies_dict[i] = n

key_strategies_1 = key_strategies.copy()
lis = list(key_strategies_dict.keys())
q = lis[0]

def remove_extra(key_stra

['balanced study routine']


In [285]:
len(key_strategies_remove) + len(key_strategies_1)

14

In [258]:
key_strategies_dict

{0: [1],
 1: [0, 5, 7, 20, 21, 25, 26],
 2: [3, 9, 17, 18, 23, 32],
 3: [2, 9, 17, 18, 23, 32],
 4: [],
 5: [1, 7, 20, 21, 25, 26],
 6: [14],
 7: [1, 5, 20, 21, 25, 26],
 8: [9],
 9: [2, 3, 8, 17, 18, 22, 23, 24, 32],
 10: [],
 11: [12],
 12: [11, 18],
 13: [14, 15],
 14: [6, 13, 15],
 15: [13, 14],
 16: [],
 17: [2, 3, 9, 18, 23, 32],
 18: [2, 3, 9, 12, 17, 23, 32],
 19: [],
 20: [1, 5, 7, 21, 25, 26],
 21: [1, 5, 7, 20, 25, 26],
 22: [9, 23, 24],
 23: [2, 3, 9, 17, 18, 22, 24, 32],
 24: [9, 22, 23],
 25: [1, 5, 7, 20, 21, 26],
 26: [1, 5, 7, 20, 21, 25],
 27: [],
 28: [],
 29: [30],
 30: [29],
 31: [33],
 32: [2, 3, 9, 17, 18, 23, 33],
 33: [31, 32]}

In [251]:
key_strategies

['balanced schedule',
 'balanced study routine',
 'consistent practice',
 'daily practice',
 'extensive reading',
 'focused study',
 'forums',
 'group study',
 'intensive course',
 'intensive speaking practice',
 'listening skill improvement',
 'mock interviews',
 'mock tests',
 'online courses',
 'online forums',
 'online resources',
 'peer feedback',
 'practice',
 'practice tests',
 'private tutor',
 'self-paced study',
 'self-study',
 'speaking clubs',
 'speaking practice',
 'speaking with native speakers',
 'structured study plan',
 'study groups',
 'time management',
 'tutoring',
 'vocabulary building',
 'vocabulary enhancement',
 'workshops',
 'writing practice',
 'writing workshops']